In [2]:
from os.path import join
import numpy as np
from scipy import optimize
from microstruktur.signal_models import three_dimensional_models
from microstruktur.signal_models import gradient_conversions

In [3]:
from importlib import reload
reload(three_dimensional_models)

ImportError: cannot import name reload

In [4]:
bvals = np.loadtxt(join(three_dimensional_models.GRADIENT_TABLES_PATH, 'bvals_hcp_wu_minn.txt'))
gradient_directions = np.loadtxt(join(three_dimensional_models.GRADIENT_TABLES_PATH, 'bvecs_hcp_wu_minn.txt'))

In [5]:
stick = three_dimensional_models.I1Stick(lambda_par=2e-3, mu=[0, 0])

In [7]:
E = stick(bvals, gradient_directions)
E

array([ 1.        ,  0.99993128,  0.99950399,  0.99497297,  0.99968034,
        0.99911933,  0.99887241,  0.99838279,  0.99764941,  0.99989114,
        0.99967153,  0.9962333 ,  0.99933009,  0.99853447,  0.99999902,
        0.99986772,  1.        ,  0.99974771,  0.9999214 ,  0.99508683,
        0.99774243,  0.99920604,  0.99858459,  0.9998681 ,  0.99779971,
        0.9967033 ,  0.99881886,  0.99861659,  0.99916206,  0.99977975,
        0.99985972,  0.99784486,  1.        ,  0.9983767 ,  0.99997567,
        0.99982919,  0.99819952,  0.99716225,  0.99936434,  0.99963177,
        0.99470837,  0.99995596,  0.99674777,  0.9995011 ,  0.99745552,
        0.99999847,  0.99916856,  0.99998948,  1.        ,  0.99872217,
        0.99964556,  0.99977166,  0.99980555,  0.9988678 ,  0.99631111,
        0.99878214,  0.9968499 ,  0.99948604,  0.99970992,  0.99856754,
        0.99769698,  0.9999972 ,  0.99720558,  0.99965142,  1.        ,
        0.99807767,  0.99735753,  0.99999862,  0.99999931,  0.99

In [8]:
x0 = stick.parameters_to_parameter_vector(mu=[np.pi/4, 0], lambda_par=2e-3)
res = optimize.minimize(
    stick.objective_function, x0, (bvals, gradient_directions, E),
    bounds=stick.bounds_for_optimization, options={'disp': True}
)
res

      fun: 1.9072338816438421e-06
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([  1.61039651e-06,   3.06367590e-06,  -4.19772164e-08])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 12
      nit: 2
   status: 0
  success: True
        x: array([  1.34283616e-03,   7.85393875e-01,   6.84412853e-08])

In [9]:
stick.parameter_vector_to_parameters(res.x)

{'lambda_par': array([ 0.00134284]),
 'mu': array([  7.85393875e-01,   6.84412853e-08])}

In [10]:
stick2 = three_dimensional_models.I1Stick(lambda_par=2e-3, mu=[np.pi/4, 0])

# Linked Parameter Example
lambda_par of the second stick is double that of the first stick

In [11]:
reload(three_dimensional_models)

<module 'microstruktur.signal_models.three_dimensional_models' from '/user/rfick/home/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/microstruktur-0.1.dev0-py2.7.egg/microstruktur/signal_models/three_dimensional_models.pyc'>

In [12]:
def double_scalar_plus_constant(x, c):
    return 2 * x + c

parameter_links = [
    (
        stick2, 'lambda_par',
        double_scalar_plus_constant, [
            (stick, 'lambda_par'),
            (None, 'partial_volume_0')
        ]
    )
]

two_sticks = three_dimensional_models.PartialVolumeCombinedMicrostrukturModel(
    [stick, stick2], parameter_links=parameter_links, 
    optimise_partial_volumes=True
)
two_sticks.parameter_cardinality

OrderedDict([('I1Stick_1_lambda_par', 1),
             ('I1Stick_2_mu', 2),
             ('I1Stick_1_mu', 2),
             ('partial_volume_0', 1)])

In [13]:
#Check if we add the linked parameter correctly
two_sticks.add_linked_parameters_to_parameters(two_sticks.parameter_defaults)

OrderedDict([('I1Stick_1_lambda_par', 0.002),
             ('I1Stick_1_mu', [0, 0]),
             ('I1Stick_2_mu', [0.7853981633974483, 0]),
             ('partial_volume_0', 0.5),
             ('I1Stick_2_lambda_par', 0.504)])

In [16]:
E = two_sticks(bvals, gradient_directions, **two_sticks.parameter_defaults)
E

array([ 1.        ,  0.93304669,  0.89447038,  0.73968382,  0.99459731,
        0.99676337,  0.73176879,  0.85181715,  0.99452009,  0.72855501,
        0.99572492,  0.74797819,  0.67487947,  0.97877696,  0.81533204,
        0.99410003,  1.        ,  0.84135576,  0.97447586,  0.7512597 ,
        0.98782721,  0.85630807,  0.96107198,  0.96529864,  0.96424104,
        0.61310906,  0.99001776,  0.70120865,  0.84961569,  0.92955813,
        0.88918847,  0.99728672,  1.        ,  0.81910816,  0.93856646,
        0.96045386,  0.86231971,  0.7869414 ,  0.89822112,  0.82371965,
        0.87719752,  0.99416528,  0.6434009 ,  0.98374955,  0.80474512,
        0.99999423,  0.96858919,  0.72041216,  1.        ,  0.92782391,
        0.73105538,  0.9971164 ,  0.80991548,  0.94551746,  0.87504245,
        0.80337041,  0.9796812 ,  0.8607821 ,  0.9882279 ,  0.99559667,
        0.89514782,  0.96603314,  0.65997492,  0.79662003,  1.        ,
        0.9251619 ,  0.69087701,  0.8415194 ,  0.89391107,  0.96

In [17]:
x0 = two_sticks.parameters_to_parameter_vector(
    **two_sticks.parameter_defaults
)
x0

array([ 0.002     ,  0.78539816,  0.        ,  0.        ,  0.        ,
        0.5       ])

In [18]:
x0 = two_sticks.parameters_to_parameter_vector(
    I1Stick_1_mu=[0, 0], I1Stick_1_lambda_par=0e-3,
    I1Stick_2_mu=[0, 0], #I1Stick_2_lambda_par=2e-3,
    partial_volume_0=.2
)
res = optimize.minimize(
    two_sticks.objective_function, x0, (bvals, gradient_directions, E),
    bounds=two_sticks.bounds_for_optimization, options={'disp': True}
)
res

      fun: 3.5323715982492766e-10
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -7.45357281e-06,   1.37380851e-06,  -4.23016701e-06,
         6.57761720e-09,  -1.05977346e-13,  -1.21545550e-08])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 196
      nit: 23
   status: 0
  success: True
        x: array([  1.98661323e-03,   7.85401077e-01,  -1.42234049e-04,
         4.76743198e-06,   7.14572426e-08,   4.99927174e-01])

In [19]:
two_sticks.parameter_vector_to_parameters(res.x)

{'I1Stick_1_lambda_par': array([ 0.00198661]),
 'I1Stick_1_mu': array([  4.76743198e-06,   7.14572426e-08]),
 'I1Stick_2_mu': array([  7.85401077e-01,  -1.42234049e-04]),
 'partial_volume_0': array([ 0.49992717])}

In [20]:
two_sticks.add_linked_parameters_to_parameters(two_sticks.parameter_vector_to_parameters(res.x))

{'I1Stick_1_lambda_par': array([ 0.00198661]),
 'I1Stick_1_mu': array([  4.76743198e-06,   7.14572426e-08]),
 'I1Stick_2_lambda_par': array([ 0.5039004]),
 'I1Stick_2_mu': array([  7.85401077e-01,  -1.42234049e-04]),
 'partial_volume_0': array([ 0.49992717])}

# Fixed Parameter Example
lambda_par of the second stick is fixed to 2e-9

In [21]:
parameter_links = [
    (
        stick2, 'lambda_par',
        lambda:2e-9, []
    )
]

two_sticks = three_dimensional_models.PartialVolumeCombinedMicrostrukturModel(
    [stick, stick2], parameter_links=parameter_links, 
    optimise_partial_volumes=True
)
two_sticks.parameter_cardinality

OrderedDict([('I1Stick_1_lambda_par', 1),
             ('I1Stick_2_mu', 2),
             ('I1Stick_1_mu', 2),
             ('partial_volume_0', 1)])

In [22]:
#Check if we add the linked parameter correctly
two_sticks.add_linked_parameters_to_parameters(two_sticks.parameter_defaults)

OrderedDict([('I1Stick_1_lambda_par', 0.002),
             ('I1Stick_1_mu', [0, 0]),
             ('I1Stick_2_mu', [0.7853981633974483, 0]),
             ('partial_volume_0', 0.5),
             ('I1Stick_2_lambda_par', 2e-09)])

In [25]:
E = two_sticks(bvals, gradient_directions, **two_sticks.parameter_defaults)
E

array([ 1.        ,  0.99996564,  0.999752  ,  0.99748648,  0.99984017,
        0.99955966,  0.99943621,  0.99919139,  0.99882471,  0.99994557,
        0.99983576,  0.99811665,  0.99966504,  0.99926724,  0.99999951,
        0.99993386,  1.        ,  0.99987386,  0.9999607 ,  0.99754341,
        0.99887121,  0.99960302,  0.9992923 ,  0.99993405,  0.99889985,
        0.99835165,  0.99940943,  0.9993083 ,  0.99958103,  0.99988987,
        0.99992986,  0.99892243,  1.        ,  0.99918835,  0.99998783,
        0.99991459,  0.99909976,  0.99858112,  0.99968217,  0.99981588,
        0.99735418,  0.99997798,  0.99837388,  0.99975055,  0.99872776,
        0.99999924,  0.99958428,  0.99999474,  1.        ,  0.99936109,
        0.99982278,  0.99988583,  0.99990278,  0.9994339 ,  0.99815555,
        0.99939107,  0.99842495,  0.99974302,  0.99985496,  0.99928377,
        0.99884849,  0.9999986 ,  0.99860279,  0.99982571,  1.        ,
        0.99903884,  0.99867876,  0.99999931,  0.99999965,  0.99

In [26]:
x0 = two_sticks.parameters_to_parameter_vector(
    **two_sticks.parameter_defaults
)
x0

array([ 0.002     ,  0.78539816,  0.        ,  0.        ,  0.        ,
        0.5       ])

In [27]:
x0 = two_sticks.parameters_to_parameter_vector(
    I1Stick_1_mu=[0, 0], I1Stick_1_lambda_par=0e-3,
    I1Stick_2_mu=[0, 0], #I1Stick_2_lambda_par=2e-3,
    partial_volume_0=.2
)
res = optimize.minimize(
    two_sticks.objective_function, x0, (bvals, gradient_directions, E),
    bounds=two_sticks.bounds_for_optimization, options={'disp': True}
)
res

      fun: 1.1685974104710362e-10
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -4.00149854e-06,   1.50701968e-15,   0.00000000e+00,
        -4.04358190e-10,  -4.16175246e-15,  -9.95149616e-08])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 21
      nit: 2
   status: 0
  success: True
        x: array([  4.95300133e-03,   0.00000000e+00,   0.00000000e+00,
         4.46024427e-07,   0.00000000e+00,   2.00113824e-01])

In [28]:
two_sticks.parameter_vector_to_parameters(res.x)

{'I1Stick_1_lambda_par': array([ 0.004953]),
 'I1Stick_1_mu': array([  4.46024427e-07,   0.00000000e+00]),
 'I1Stick_2_mu': array([ 0.,  0.]),
 'partial_volume_0': array([ 0.20011382])}

In [29]:
two_sticks.add_linked_parameters_to_parameters(two_sticks.parameter_vector_to_parameters(res.x))

{'I1Stick_1_lambda_par': array([ 0.004953]),
 'I1Stick_1_mu': array([  4.46024427e-07,   0.00000000e+00]),
 'I1Stick_2_lambda_par': 2e-09,
 'I1Stick_2_mu': array([ 0.,  0.]),
 'partial_volume_0': array([ 0.20011382])}

In [30]:
%timeit lambda:2

10000000 loops, best of 3: 55.2 ns per loop


In [31]:
%timeit 2

100000000 loops, best of 3: 9.26 ns per loop
